Using the nwb_photostim package

In [1]:
from file_classes import SpatialLightModulator, PhotostimulationDevice, HolographicPattern, PhotostimulationSeries, StimulusPresentation
import numpy as np

## Spatial light modulator

In [2]:
slm = SpatialLightModulator(name="slm", dimensions=np.array([1, 2, 3]))

## PhotostimulationDevice

In [ ]:
photostim_dev = PhotostimulationDevice(name="photostim_dev", description="photostim_device", type='LED',
                                               wavelength=320, slm=slm)

## Holographic stimulation pattern

In [4]:
mask_roi = np.zeros((50, 50))
x = np.random.randint(0, 3)
y = np.random.randint(0, 3)
mask_roi[x:x + 5, y:y + 5] = 1

hp = HolographicPattern(name='HolographicPattern', mask_roi=mask_roi, stimulation_diameter=5)

In [9]:
pixel_roi = []

for i in range(5):
    x = np.random.randint(0, 95)
    y = np.random.randint(0, 95)

    for ix in range(x, x + 1):
        for iy in range(y, y + 1):
            pixel_roi.append((ix, iy))

hp = HolographicPattern(name='HolographicPattern', pixel_roi=pixel_roi, stimulation_diameter=8, dimension=[100, 100])
mask_roi = hp.pixel_to_mask_roi()

## Stimulation series


### Interval format

In [ ]:
photostim_series = PhotostimulationSeries(name="photosim series", holographic_pattern=hp, unit='SIunit', data=[1, -1, 1, -1], timestamps=[0.5, 1, 2, 4])

In [1]:
empty_series = PhotostimulationSeries(name="photosim series", holographic_pattern=hp, unit='SIunit')

NameError: name 'PhotostimulationSeries' is not defined

In [ ]:
empty_series.add_interval(10, 20)
empty_series.add_interval(30, 40)

### Series format

In [ ]:
photostim_series = PhotostimulationSeries(name="photosim series", holographic_pattern=hp, unit='SIunit', format='series', stimulus_duration = 0.05, data=[0, 0, 0, 1, 1, 0], timestamps=[0, 0.5, 1, 1.5, 3, 6])

In [ ]:
photostim_series = PhotostimulationSeries(name="photosim series", holographic_pattern=hp, unit='SIunit', format='series', data=[0, 0, 0, 1, 1, 0], rate=10.)
photostim_series.add_presentation(timestamp=2.5)
print(photostim_series.to_dataframe())

## PhotostimulationTable

In [ ]:
sp = StimulusPresentation(name='test', description='test desc')
sp.add_event_type(label='stim1', stimulus_description='d1', photostimulation_series=photostim_series, pattern=hp)

Displaying table

Plotting stimulation intervals

## Writing the file

## Reading the file